In [1]:
import yfinance as yf
import pandas as pd
import random
import numpy as np
import random

from datetime import datetime as dt
from workflow import Indicators

In [6]:
stocks = ["AAPL", "JPM", "DIS"]

len(stocks)

3

State Space:
- $B_{t}$, Portfolio Balance
- $V_{t}$, Turbulence Index
- $P_{t}$, Stock Prices
- $M_{t}$, Moving-Average Convergence Divergence (MACD) Signal
- $H_{t}$, MACD Histogram
- $R_{t}$, Relative Strength Index (RSI)
- $C_{t}$, Commodity Channel Index (CCI)
- $X_{t}$, Average Directional Index

State Space:
- $\bigl[B_{t}\bigr] + \bigl[V_{t}\bigr] + \bigl[\text{Number of Stocks} \times \bigl[S_{t},\text{ }P_{t},\text{ }M_{t},\text{ }H_{t},\text{ }R_{t},\text{ }C_{t},\text{ }X_{t}\bigr]\bigr]$

In [16]:
stock_dim = 3
feature_dim = 5
account_dim = 2
balance_dim = 1
turbulence_dim = 1

state_dim = balance_dim + turbulence_dim + stock_dim * (feature_dim + account_dim)
state_dim

23

In [8]:
start = "2009-01-01"
end = dt.now()

stock_data = yf.download(stocks, start, end).dropna(axis=1)
stock_prices = stock_data["Adj Close"]

f"{len(stock_prices.columns)} stocks", \
    f"{round((stock_prices.index.max() - stock_prices.index.min()).days / 365, 2)}Y"

[*********************100%%**********************]  3 of 3 completed


('3 stocks', '14.87Y')

In [9]:
indicators = Indicators(stock_data)
features = indicators.get()

features.head()

,AAPL_M,DIS_M,JPM_M,AAPL_H,DIS_H,JPM_H,AAPL_R,DIS_R,JPM_R,AAPL_C,DIS_C,JPM_C,AAPL_X,DIS_X,JPM_X,TURB
Date,,,,,,,,,,,,,,,,
2009-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,0.009250,-0.028850,-0.116050,0.007400,-0.023080,-0.092840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,0.012624,0.004515,-0.170835,0.008619,0.008228,-0.118100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,0.010269,-0.047017,-0.310731,0.005011,-0.034643,-0.206396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,0.012389,-0.106089,-0.464873,0.005705,-0.074972,-0.288431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
s = pd.concat([stock_prices, features], axis=1).dropna()

s.shape, s.head()

((3693, 19),
                 AAPL        DIS        JPM    AAPL_M     DIS_M     JPM_M  \
 Date                                                                       
 2009-03-17  3.017134  15.344792  17.415800  0.028964 -0.496127 -0.220801   
 2009-03-18  3.073445  15.594510  18.780525  0.047260 -0.385191  0.094233   
 2009-03-19  3.076473  15.241461  17.284174  0.061297 -0.322049  0.220614   
 2009-03-20  3.075564  15.026186  16.037222  0.071524 -0.286082  0.217644   
 2009-03-23  3.259328  16.292004  19.992840  0.093380 -0.153666  0.528384   
 
               AAPL_H     DIS_H     JPM_H     AAPL_R      DIS_R      JPM_R  \
 Date                                                                        
 2009-03-17  0.034247  0.259793  0.561729  62.816630  53.582561  58.959516   
 2009-03-18  0.042034  0.296583  0.701410  68.653609  60.064399  60.315013   
 2009-03-19  0.044857  0.287780  0.662233  68.634656  57.120986  54.852097   
 2009-03-20  0.044067  0.258998  0.527410  71.536805  61

In [11]:
s.to_csv("prices_and_features.csv")

In [15]:
(len(s.columns[len(stocks):]) - feature_dim) // len(stocks)

3